In [1]:
import os
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from stable_baselines3 import PPO
from enum import Enum
import talib
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
import time
import hmac
import hashlib
import requests
import numpy
import json

import firebase_admin
from firebase_admin import credentials, firestore
from google.cloud.firestore_v1.base_query import FieldFilter

In [2]:
model = PPO.load('saved_models/best_model_800000_LSTM4.zip')

In [3]:
class Positions(int, Enum):
    SHORT = 0
    LONG = 1
    HOLD = 2
    TAKE = 3

In [4]:
def getTickerData(ticker, period, interval):
    hist = yf.download(tickers=ticker, period=period, interval=interval)
    df = pd.DataFrame(hist)
    df = df.reset_index()
    return df

In [5]:
def calculate_percentage_increase(final_value, starting_value):
    try:
        return 100 * ((final_value - starting_value) / starting_value)
    except:
        print(final_value, starting_value)

In [6]:
def supertrend_indicator(df, atr_period, multiplier):
    
    high = df['high']
    low = df['low']
    close = df['close']
    
    price_diffs = [high - low, 
                   high - close.shift(), 
                   close.shift() - low]
    true_range = pd.concat(price_diffs, axis=1)
    true_range = true_range.abs().max(axis=1)
    atr = true_range.ewm(alpha=1/atr_period,min_periods=atr_period).mean() 
    hl2 = (high + low) / 2
    final_upperband = upperband = hl2 + (multiplier * atr)
    final_lowerband = lowerband = hl2 - (multiplier * atr)
    
    supertrend = [True] * len(df)
    
    for i in range(1, len(df.index)):
        curr, prev = i, i-1
        
        if close[curr] > final_upperband[prev]:
            supertrend[curr] = True
        elif close[curr] < final_lowerband[prev]:
            supertrend[curr] = False
        else:
            supertrend[curr] = supertrend[prev]

            if supertrend[curr] == True and final_lowerband[curr] < final_lowerband[prev]:
                final_lowerband[curr] = final_lowerband[prev]
            if supertrend[curr] == False and final_upperband[curr] > final_upperband[prev]:
                final_upperband[curr] = final_upperband[prev]

        if supertrend[curr] == True:
            final_upperband[curr] = np.nan
        else:
            final_lowerband[curr] = np.nan
    
    return final_lowerband, final_upperband

In [7]:
def rsi_indicator(df):
    rsi = talib.RSI(df.close, 14)

    signals = []
    for i in range(0, len(rsi)):
        if rsi[i] > 60: # Default value: 70
            signals.append(Positions.SHORT)
        elif rsi[i] < 40: # Default value: 30
            signals.append(Positions.LONG)
        else:
            signals.append(Positions.HOLD)
        
    buy_signal = [True if signals[n]==1 else False for n in range(0, len(signals))]
    sell_signal = [True if signals[n]==-1 else False for n in range(0, len(signals))]
    
    return signals, buy_signal, sell_signal

In [8]:
def swing_detection(index, df):
    sh = []
    sl = []
    start = (index*2) - 1
    for i in range(index-1):
        sh.append(False)
        sl.append(False)
    for ci, row in df.iterrows():
        
        swing_high = False
        swing_low = False
        
        if ci < start:
            continue
        
        swing_point_high = df['high'][ci - index]
        swing_point_low = df['low'][ci - index]
        
        for i in range(0, start):
            swing_high = True
            if i < index:
                if df['high'][ci - i] > swing_point_high:
                    swing_high = False
                    break
            if i > index:
                if df['high'][ci - i] >= swing_point_high:
                    swing_high = False
                    break
            
        for i in range(0, start):
            swing_low = True
            if i < index:
                if df.low[ci - i] < swing_point_low: 
                    swing_low = False
                    break  
            if i > index:
                if df.low[ci - i] <= swing_point_low: 
                    swing_low = False
                    break 
            
        sh.append(swing_high)
        sl.append(swing_low)
        
    for i in range(index):
        sh.append(False)
        sl.append(False)
        
    current_sh = 0
    current_sl = 0
    sh_nums = []
    sl_nums = []
    for i, row in df.iterrows():
        if sh[i] == True:
            current_sh = df.high[i]
        if sl[i] == True:
            current_sl = df.low[i]
        sh_nums.append(current_sh)
        sl_nums.append(current_sl)
    return sh, sl, sh_nums, sl_nums

In [9]:
def produce_prediction(df, window):  
    prediction = (df.shift(window)['close'] <= df['close'])
    
    return prediction.astype(int)

In [10]:
def preprocess_data(df):
    scaler = MinMaxScaler()
    
    mfi_indicator = talib.MFI(df['high'], df['low'], df['close'], df['volume'])
    close_b = produce_prediction(df, 1)
    rsi_signals, _, _ = rsi_indicator(df)
    sh, sl, sh_nums, sl_nums = swing_detection(5, df)    
    final_lowerband, final_upperband = supertrend_indicator(df, 10, 3)
    fu_modified = [Positions.LONG if not np.isnan(lowerband) else Positions.SHORT for lowerband in final_lowerband]

    df['close_binary'] = close_b
    df['mfi'] = mfi_indicator
    df['sh_nums'] = sh_nums
    df['sl_nums'] = sl_nums
    df['supertrend'] = fu_modified
    df['rsi_signals'] = rsi_signals
    
    df = df.drop(columns={"volume", "Adj Close", "open", "high", "low", "date"})
    df = df.dropna()
    
    df[['mfi']] = scaler.fit_transform(df[['mfi']])
    
    return df

In [27]:
df = getTickerData('btc-usd', 'max', '1d')
df = df.rename(columns={'Close': 'close', 'Open': 'open', 'High': 'high', 'Low': 'low', 'Volume': 'volume', 'Datetime': 'date', 'Date': 'date'})
df = preprocess_data(df)
df = df[0:df.shape[0]-1]

[*********************100%***********************]  1 of 1 completed


In [12]:
cred = credentials.Certificate("serviceAccountKey.json")
firebase_admin.initialize_app(cred)

In [13]:
db = firestore.client()

In [14]:
# getting info from database
docs = db.collection("agents").where(filter=FieldFilter("position", "==", 'hold')).stream()
agents = []

for doc in docs:
    agents.append(doc.to_dict())

In [28]:
# getting prediction for current market data
prediction = model.predict(df[['mfi', 'close_binary', 'supertrend', 'rsi_signals']].tail(89))[0]
position_to_take = 'hold'
if prediction == Positions.LONG:
    position_to_take = 'BUY'
elif prediction == Positions.SHORT:
    position_to_take = 'SELL'

In [45]:
# Checking if it's valid to trade

btc_last_price = df['close'].iloc[-1]
swing_low = df['sl_nums'].iloc[-1]
swing_high = df['sh_nums'].iloc[-1]
is_valid_to_trade = False
if position_to_take != 'hold':
    is_valid_to_trade = True
if position_to_take == 'BUY': 
    if btc_last_price > swing_low:
        is_valid_to_trade = True
if position_to_take == 'SELL': 
    if btc_last_price < swing_high:
        is_valid_to_trade = True
print("Is it valid to trade: ", is_valid_to_trade)

Is it valid to trade:  True


In [23]:
HOST_URL = 'https://testnet.binancefuture.com'
END_POINT = '/fapi/v1/order'

if is_valid_to_trade:    
    for i, agent in enumerate(agents):
        if agent['associatedAccountName'] == 'asd2':
            continue
        
        # Setting up request info
        api_key = agent['apiKey']
        api_secret = agent['apiSecret']
        amount_of_usd_to_use = agent['usdtToUse'] * (agent['percentagePerTrade'] * 0.01)
        converted_quantity = amount_of_usd_to_use / btc_last_price
        converted_quantity = round(float(converted_quantity.iloc[0]), 3)
        data_query= f"symbol=BTCUSDT&side={position_to_take}&type=MARKET&quantity={converted_quantity}&timestamp={str(time.time()).replace('.','')[0: 13]}"
        signed_hash_key = hmac.new(
            bytes(api_secret, 'latin-1'),
            msg=bytes(data_query, 'latin-1'),
            digestmod=hashlib.sha256
        ).hexdigest()
        
        data_query= data_query + f"&signature={signed_hash_key}"
        url = HOST_URL + END_POINT + "?" + data_query
        
        # Sending request
        header = {'X-MBX-APIKEY': api_key, 'signature': signed_hash_key}
        response = requests.post(url=url, headers=header)
        print(response.text)

symbol=BTCUSDT&side=BUY&type=MARKET&quantity=0.029&timestamp=1694097822014
{"orderId":3459435062,"symbol":"BTCUSDT","status":"NEW","clientOrderId":"3UFrowqj10L2PCfja9LXDv","price":"0.00","avgPrice":"0.00","origQty":"0.029","executedQty":"0.000","cumQty":"0.000","cumQuote":"0.00000","timeInForce":"GTC","type":"MARKET","reduceOnly":false,"closePosition":false,"side":"BUY","positionSide":"BOTH","stopPrice":"0.00","workingType":"CONTRACT_PRICE","priceProtect":false,"origType":"MARKET","priceMatch":"NONE","selfTradePreventionMode":"NONE","goodTillDate":0,"updateTime":1694097823677}
